**['source': 'Deep Learning with Python Francois Cholltet' ]**

We will train GAN on CIFAR10, a dataset of 50000 32x32 RGB images belonging to 10 classes (5000 images per class). From this dataset, we will only be using images belonging to class "Frog".

1. A generator network maps vectors of shape (latent_dim,) to images of shape (32,32,3).
2. A discriminator network maps images of shape (32,32,3) to a binary score estimating th eprobability that image is real.
3. A GAN network chains the generator and discriminator together: gan(x) = discriminator(generator(x))
4. We train discriminator using examples of real and fake images aling with real/fake labels
5. To train generator, we use gradients of generator's weights with regard to loss of gan model.This means every step you move weights of genrator in direction that makes dicriminator more likely to classify as real images decoded by generator. In other words, you train genrator to fool the discriminator.

Tricks used in GAN
1. tanh is used as last activation function instead of sigmoid.
2. we sample points from latent space using a normal distribution.
3. Since sparse gradients are a problem and these are caused by max pooling and ReLU, we use LeakyReLU instead of relu, and instead of mapooling we use strided convolutions.
4. we also use kernel size thats divisible by stride size in conv2d layers.

### GAN Network

In [1]:
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy as np
latent_dim = 32
height = 32
width = 32
channels = 3

32768

In [8]:
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels,7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 256)       1048

#### Discriminator

In [11]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 6, 6, 128)         0   

In [12]:
discriminator_optimizer = keras.optimizers.RMSprop(
 lr=0.0008,
 clipvalue=1.0,
 decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
loss='binary_crossentropy')

#### Adversarial Network

In [13]:
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

#### How to train DCGAN
1. draw random points from latent space
2. generate images with generator
3. mix generated images with real ones
4. discriminator is trained using mix images with targets stating if image is real or fake similar to image classification.
5. draw new random points from latent space
6. train gan using these random vectors,with targets that all say these are real images. this updates weight of generator( as discriminator is frozen inside "gan") to move them toward getting discriminator to predict these are real images for generatoed images.This trains generator to fool the discriminator

In [14]:
import os
from tensorflow.keras.preprocessing import image
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

In [18]:
x_train.shape

(50000, 32, 32, 3)

In [19]:
x_train = x_train[y_train.flatten() == 6]
x_train = x_train.reshape((x_train.shape[0],) +(height, width, channels)).astype('float32') / 255.
iterations = 10000
batch_size = 20

In [20]:

save_dir = './gen_images'

In [ ]:
start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
    latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)),
    np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    d_loss = discriminator.train_on_batch(combined_images, labels)
    random_latent_vectors = np.random.normal(size=(batch_size,
    latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors,
    misleading_targets)
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    if step % 100 == 0:
        gan.save_weights('gan.h5')
    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)
    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,
     'generated_frog' + str(step) + '.png'))
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,
     'real_frog' + str(step) + '.png'))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
discriminator loss: 0.6792344
adversarial loss: 0.65956837
discriminator loss: 7.310805
adversarial loss: 12.347681
discriminator loss: 2.7191699
adversarial loss: 4.3525916e-10
discriminator loss: 12.682602
adversarial loss: 0.854418
discriminator loss: 0.7954281
adversarial loss: 1.0237603
discriminator loss: 0.7609694
adversarial loss: 1.1666873
discriminator loss: 0.6688085
adversarial loss: 1.0287492
discriminator loss: 0.4664896
adversarial loss: 6.478769
discriminator loss: 0.28774992
adversarial loss: 5.317338
discriminator loss: 0.22238359
adversarial loss: 7.6

discriminator loss: 0.6627587
adversarial loss: 0.78583026
discriminator loss: 0.7502017
adversarial loss: 0.9094402
discriminator loss: 0.6824646
adversarial loss: 0.70203483
discriminator loss: 0.6735884
adversarial loss: 1.3515283
discriminator loss: 0.76832116
adversarial loss: 0.94468975
discriminator loss: 0.8304
adversarial loss: 0.925439
discriminator loss: 0.70254976
adversarial loss: 0.7646022
discriminator loss: 0.6713743
adversarial loss: 0.8666111
discriminator loss: 0.69796
adversarial loss: 0.79236513
discriminator loss: 0.6699467
adversarial loss: 0.80447924
discriminator loss: 0.69500005
adversarial loss: 0.8114251
discriminator loss: 0.65257597
adversarial loss: 0.969581
discriminator loss: 0.65607566
adversarial loss: 1.0468199
discriminator loss: 0.68816805
adversarial loss: 0.71204156
discriminator loss: 0.72489405
adversarial loss: 1.6752994
discriminator loss: 0.78468287
adversarial loss: 0.95245516
discriminator loss: 0.72287524
adversarial loss: 0.72057706
disc

discriminator loss: 0.7144278
adversarial loss: 0.72312963
discriminator loss: 0.7023387
adversarial loss: 0.84362936
discriminator loss: 0.72190714
adversarial loss: 0.7995756
discriminator loss: 0.6876413
adversarial loss: 0.7035942
discriminator loss: 0.72204983
adversarial loss: 0.73989266
discriminator loss: 0.70181876
adversarial loss: 0.80262697
discriminator loss: 0.68023866
adversarial loss: 0.8183529
discriminator loss: 0.7041205
adversarial loss: 0.7392777
discriminator loss: 0.82229626
adversarial loss: 0.7372409
discriminator loss: 0.7306556
adversarial loss: 0.7705339
discriminator loss: 0.71417856
adversarial loss: 0.74705493
discriminator loss: 0.69494236
adversarial loss: 0.75767994
discriminator loss: 0.70527375
adversarial loss: 0.7347314
discriminator loss: 0.6984216
adversarial loss: 0.71911657
discriminator loss: 0.7054789
adversarial loss: 0.731788
discriminator loss: 0.69137895
adversarial loss: 0.7319962
discriminator loss: 0.6939454
adversarial loss: 0.7348605

discriminator loss: 0.7054849
adversarial loss: 0.754414
discriminator loss: 0.6937227
adversarial loss: 0.7370683
discriminator loss: 0.69910246
adversarial loss: 0.74440306
discriminator loss: 0.7017036
adversarial loss: 0.7645494
discriminator loss: 0.7003691
adversarial loss: 0.7378856
discriminator loss: 0.6914607
adversarial loss: 0.75404507
discriminator loss: 0.69861925
adversarial loss: 0.7554132
discriminator loss: 0.7004522
adversarial loss: 0.7412085
discriminator loss: 0.70225513
adversarial loss: 0.7548471
discriminator loss: 0.69143593
adversarial loss: 0.74533427
discriminator loss: 0.6968362
adversarial loss: 0.7484413
discriminator loss: 0.69230115
adversarial loss: 0.76200056
discriminator loss: 0.6901235
adversarial loss: 0.7779864
discriminator loss: 0.6942252
adversarial loss: 0.7641853
discriminator loss: 0.70903254
adversarial loss: 0.7714244
discriminator loss: 0.70455045
adversarial loss: 0.73445994
discriminator loss: 0.689572
adversarial loss: 0.7356287
disc

discriminator loss: 0.6953038
adversarial loss: 0.7441853
discriminator loss: 0.72432935
adversarial loss: 0.7724645
discriminator loss: 0.6986152
adversarial loss: 0.7397791
discriminator loss: 0.68546
adversarial loss: 0.7474282
discriminator loss: 0.6899079
adversarial loss: 0.7080139
discriminator loss: 0.68452466
adversarial loss: 0.7227043
discriminator loss: 0.68069327
adversarial loss: 0.6865204
discriminator loss: 0.7059292
adversarial loss: 0.7558173
discriminator loss: 0.6998162
adversarial loss: 0.78722066
discriminator loss: 0.6919589
adversarial loss: 0.78713536
discriminator loss: 0.70102954
adversarial loss: 0.7423897
discriminator loss: 0.6827978
adversarial loss: 0.76346767
discriminator loss: 0.68874156
adversarial loss: 0.7232992
discriminator loss: 0.6852678
adversarial loss: 0.7261589
discriminator loss: 0.70845526
adversarial loss: 0.72443426
discriminator loss: 0.6996087
adversarial loss: 0.7725633
discriminator loss: 0.7074145
adversarial loss: 0.7396368
discri

discriminator loss: 0.72628236
adversarial loss: 0.73536575
discriminator loss: 0.9085986
adversarial loss: 0.7851194
discriminator loss: 0.7091586
adversarial loss: 0.7384864
discriminator loss: 0.6865903
adversarial loss: 0.74852574
discriminator loss: 0.68120235
adversarial loss: 0.7630739
discriminator loss: 0.69177026
adversarial loss: 0.7651444
discriminator loss: 0.7028078
adversarial loss: 0.7411774
discriminator loss: 0.707589
adversarial loss: 0.7247606
discriminator loss: 0.69905496
adversarial loss: 0.7150698
discriminator loss: 0.7049124
adversarial loss: 0.72913027
discriminator loss: 0.6939896
adversarial loss: 0.73175913
discriminator loss: 0.6917943
adversarial loss: 0.74949455
discriminator loss: 0.69158095
adversarial loss: 0.7303859
discriminator loss: 0.6818473
adversarial loss: 0.71693695
discriminator loss: 0.6880037
adversarial loss: 0.72636503
discriminator loss: 0.69601536
adversarial loss: 0.7202867
discriminator loss: 0.692978
adversarial loss: 0.7765009
dis

discriminator loss: 0.69269836
adversarial loss: 0.7627184
discriminator loss: 0.6988974
adversarial loss: 0.74663275
discriminator loss: 0.694626
adversarial loss: 0.751597
discriminator loss: 0.6877221
adversarial loss: 0.76184946
discriminator loss: 0.70490086
adversarial loss: 0.7314276
discriminator loss: 0.6971985
adversarial loss: 0.7342974
discriminator loss: 0.6999624
adversarial loss: 0.7450234
discriminator loss: 0.6959443
adversarial loss: 0.7484879
discriminator loss: 0.6937251
adversarial loss: 0.7404486
discriminator loss: 0.6889215
adversarial loss: 0.73998255
discriminator loss: 0.69535273
adversarial loss: 0.7336282
discriminator loss: 0.6951698
adversarial loss: 0.745391
discriminator loss: 0.6982787
adversarial loss: 0.7501549
discriminator loss: 0.69429237
adversarial loss: 0.7499421
discriminator loss: 0.69452256
adversarial loss: 0.7463513
discriminator loss: 0.6960497
adversarial loss: 0.75334835
discriminator loss: 0.698681
adversarial loss: 0.7416441
discrimin